# Sensitivity Analysis

**Purpose**: Test directionality - does changing X affect Y in expected ways?

**This is a throwaway notebook** - for SCRI validation session only.

---

In [ ]:
# Import from existing engine (READ-ONLY)
import sys
sys.path.insert(0, '../..')

from seleensim.entities import Site, Trial, PatientFlow
from seleensim.distributions import Triangular, Gamma, Bernoulli
from seleensim.simulation import SimulationEngine
from seleensim.constraints import (
    BudgetThrottlingConstraint,
    ResourceCapacityConstraint,
    LinearResponseCurve,
    LinearCapacityDegradation
)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Baseline trial (same as notebook 01)
site = Site(
    site_id="SITE_001",
    activation_time=Triangular(low=30, mode=45, high=90),
    enrollment_rate=Gamma(shape=2, scale=1.5),
    dropout_rate=Bernoulli(p=0.15)
)

flow = PatientFlow(
    flow_id="STANDARD_FLOW",
    states={"enrolled", "completed"},
    initial_state="enrolled",
    terminal_states={"completed"},
    transition_times={
        ("enrolled", "completed"): Triangular(low=90, mode=180, high=365)
    }
)

trial = Trial(
    trial_id="BASELINE_TRIAL",
    target_enrollment=200,
    sites=[site],
    patient_flow=flow
)

print("Trial configured")

## Test 1: Budget min_speed_ratio Sweep

**Question**: Does increasing max slowdown increase completion time?

**Expected**: More slowdown → Longer completion time

In [ ]:
# Sweep min_speed_ratio from 0.2 (5x slowdown) to 1.0 (no slowdown)
min_speed_ratios = [0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0]
budget_per_day = 50000  # Fixed

results_sweep = []

print("Running sensitivity sweep...")
for ratio in min_speed_ratios:
    constraint = BudgetThrottlingConstraint(
        budget_per_day=budget_per_day,
        response_curve=LinearResponseCurve(min_speed_ratio=ratio)
    )
    
    engine = SimulationEngine(master_seed=42, constraints=[constraint])
    result = engine.run(trial, num_runs=100)
    
    results_sweep.append({
        'min_speed_ratio': ratio,
        'max_slowdown': 1/ratio,
        'p50_days': result.completion_time_p50,
        'p90_days': result.completion_time_p90
    })
    print(f"  {ratio:.1f} ({1/ratio:.1f}x slowdown) → P50={result.completion_time_p50:.1f} days")

df = pd.DataFrame(results_sweep)
print("\nComplete results:")
print(df.to_string(index=False))

In [ ]:
# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: min_speed_ratio vs completion time
axes[0].plot(df['min_speed_ratio'], df['p50_days'], 'o-', label='P50', linewidth=2)
axes[0].plot(df['min_speed_ratio'], df['p90_days'], 's-', label='P90', linewidth=2)
axes[0].set_xlabel('min_speed_ratio')
axes[0].set_ylabel('Completion Time (days)')
axes[0].set_title('Budget Constraint: Speed Ratio vs Completion Time')
axes[0].legend()
axes[0].grid(alpha=0.3)
axes[0].invert_xaxis()  # Lower ratio = more slowdown

# Plot 2: max_slowdown vs completion time
axes[1].plot(df['max_slowdown'], df['p50_days'], 'o-', label='P50', linewidth=2)
axes[1].plot(df['max_slowdown'], df['p90_days'], 's-', label='P90', linewidth=2)
axes[1].set_xlabel('Max Slowdown (x)')
axes[1].set_ylabel('Completion Time (days)')
axes[1].set_title('Budget Constraint: Max Slowdown vs Completion Time')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

**Ask SCRI**:
- Does this direction make sense?
- Is the magnitude of change reasonable?
- Where on this curve is your typical trial?

## Test 2: Budget Daily Rate Sweep

**Question**: Does increasing budget decrease completion time?

**Expected**: More budget → Less pressure → Shorter completion time

In [ ]:
# Sweep budget_per_day
budgets = [25000, 50000, 75000, 100000, 150000]
min_speed_ratio = 0.5  # Fixed at 2x max slowdown

results_budget = []

print("Running budget sweep...")
for budget in budgets:
    constraint = BudgetThrottlingConstraint(
        budget_per_day=budget,
        response_curve=LinearResponseCurve(min_speed_ratio=min_speed_ratio)
    )
    
    engine = SimulationEngine(master_seed=42, constraints=[constraint])
    result = engine.run(trial, num_runs=100)
    
    results_budget.append({
        'budget_per_day': budget,
        'p50_days': result.completion_time_p50,
        'p90_days': result.completion_time_p90
    })
    print(f"  ${budget:,}/day → P50={result.completion_time_p50:.1f} days")

df_budget = pd.DataFrame(results_budget)
print("\nComplete results:")
print(df_budget.to_string(index=False))

In [ ]:
# Visualize
plt.figure(figsize=(10, 5))
plt.plot(df_budget['budget_per_day'], df_budget['p50_days'], 'o-', label='P50', linewidth=2)
plt.plot(df_budget['budget_per_day'], df_budget['p90_days'], 's-', label='P90', linewidth=2)
plt.xlabel('Daily Budget ($)')
plt.ylabel('Completion Time (days)')
plt.title('Budget Constraint: Daily Budget vs Completion Time')
plt.legend()
plt.grid(alpha=0.3)
plt.ticklabel_format(style='plain', axis='x')
plt.show()

**Ask SCRI**:
- Does this make sense?
- What's a typical daily budget for your trials?
- Where does the curve flatten out?

## Test 3: Capacity Threshold Sweep

**Question**: Does starting degradation earlier increase completion time?

**Expected**: Earlier degradation → More slowdown → Longer completion time

In [ ]:
# Sweep capacity threshold
thresholds = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
max_multiplier = 2.0  # Fixed
max_utilization = 1.5  # Fixed

results_capacity = []

print("Running capacity threshold sweep...")
for threshold in thresholds:
    capacity_response = LinearCapacityDegradation(
        threshold=threshold,
        max_multiplier=max_multiplier,
        max_utilization=max_utilization
    )
    
    constraint = ResourceCapacityConstraint(
        resource_id="CRA",
        capacity_response=capacity_response
    )
    
    engine = SimulationEngine(master_seed=42, constraints=[constraint])
    result = engine.run(trial, num_runs=100)
    
    results_capacity.append({
        'threshold': threshold,
        'p50_days': result.completion_time_p50,
        'p90_days': result.completion_time_p90
    })
    print(f"  Threshold={threshold*100:.0f}% → P50={result.completion_time_p50:.1f} days")

df_capacity = pd.DataFrame(results_capacity)
print("\nComplete results:")
print(df_capacity.to_string(index=False))

In [ ]:
# Visualize
plt.figure(figsize=(10, 5))
plt.plot(df_capacity['threshold'], df_capacity['p50_days'], 'o-', label='P50', linewidth=2)
plt.plot(df_capacity['threshold'], df_capacity['p90_days'], 's-', label='P90', linewidth=2)
plt.xlabel('Degradation Threshold (utilization %)')
plt.ylabel('Completion Time (days)')
plt.title('Capacity Constraint: Threshold vs Completion Time')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

**Ask SCRI**:
- Does this match your experience?
- At what utilization % do you see slowdowns start?
- Is the effect size realistic?

## Test 4: Capacity Max Multiplier Sweep

**Question**: Does worse degradation increase completion time?

**Expected**: Higher max multiplier → More severe slowdown → Longer completion time

In [ ]:
# Sweep max_multiplier
multipliers = [1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0]
threshold = 0.8  # Fixed
max_utilization = 1.5  # Fixed

results_multiplier = []

print("Running max multiplier sweep...")
for mult in multipliers:
    capacity_response = LinearCapacityDegradation(
        threshold=threshold,
        max_multiplier=mult,
        max_utilization=max_utilization
    )
    
    constraint = ResourceCapacityConstraint(
        resource_id="CRA",
        capacity_response=capacity_response
    )
    
    engine = SimulationEngine(master_seed=42, constraints=[constraint])
    result = engine.run(trial, num_runs=100)
    
    results_multiplier.append({
        'max_multiplier': mult,
        'p50_days': result.completion_time_p50,
        'p90_days': result.completion_time_p90
    })
    print(f"  Max {mult:.1f}x slower → P50={result.completion_time_p50:.1f} days")

df_multiplier = pd.DataFrame(results_multiplier)
print("\nComplete results:")
print(df_multiplier.to_string(index=False))

In [ ]:
# Visualize
plt.figure(figsize=(10, 5))
plt.plot(df_multiplier['max_multiplier'], df_multiplier['p50_days'], 'o-', label='P50', linewidth=2)
plt.plot(df_multiplier['max_multiplier'], df_multiplier['p90_days'], 's-', label='P90', linewidth=2)
plt.xlabel('Max Degradation Multiplier (x)')
plt.ylabel('Completion Time (days)')
plt.title('Capacity Constraint: Max Multiplier vs Completion Time')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

**Ask SCRI**:
- What's the worst slowdown you've seen?
- 2x? 3x? 5x?
- Does this curve shape match reality?

## Summary Table: All Sensitivity Tests

In [ ]:
print("\n=== SENSITIVITY ANALYSIS SUMMARY ===")
print("\nTest 1: Budget min_speed_ratio")
print(f"  Range: {df['min_speed_ratio'].min():.1f} to {df['min_speed_ratio'].max():.1f}")
print(f"  P50 impact: {df['p50_days'].min():.1f} to {df['p50_days'].max():.1f} days")
print(f"  Delta: {df['p50_days'].max() - df['p50_days'].min():.1f} days")

print("\nTest 2: Budget per day")
print(f"  Range: ${df_budget['budget_per_day'].min():,} to ${df_budget['budget_per_day'].max():,}")
print(f"  P50 impact: {df_budget['p50_days'].min():.1f} to {df_budget['p50_days'].max():.1f} days")
print(f"  Delta: {df_budget['p50_days'].max() - df_budget['p50_days'].min():.1f} days")

print("\nTest 3: Capacity threshold")
print(f"  Range: {df_capacity['threshold'].min()*100:.0f}% to {df_capacity['threshold'].max()*100:.0f}%")
print(f"  P50 impact: {df_capacity['p50_days'].min():.1f} to {df_capacity['p50_days'].max():.1f} days")
print(f"  Delta: {df_capacity['p50_days'].max() - df_capacity['p50_days'].min():.1f} days")

print("\nTest 4: Capacity max multiplier")
print(f"  Range: {df_multiplier['max_multiplier'].min():.1f}x to {df_multiplier['max_multiplier'].max():.1f}x")
print(f"  P50 impact: {df_multiplier['p50_days'].min():.1f} to {df_multiplier['p50_days'].max():.1f} days")
print(f"  Delta: {df_multiplier['p50_days'].max() - df_multiplier['p50_days'].min():.1f} days")

---

## SCRI Validation Questions

For each sensitivity test:

1. **Direction correct?** Does increasing X increase/decrease Y as expected?
2. **Magnitude realistic?** Is the size of the effect reasonable?
3. **Shape intuitive?** Linear? Threshold? Something else?
4. **Parameter range right?** Are we testing the right range of values?

**If answers are YES**: Architecture validated for this parameter

**If answers are NO**: Document why - this reveals gaps in the model